In [104]:
num_labels = 19

In [105]:
from transformers import BertTokenizer, AdamW, BertModel, BertPreTrainedModel

from transformers import AutoConfig

from tqdm import tqdm

config = AutoConfig.from_pretrained("bert-base-multilingual-cased")

# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertModel.from_pretrained("bert-base-multilingual-cased")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

In [109]:
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from transformers import BertModel,BertPreTrainedModel

class BertSoftmaxForNer(BertPreTrainedModel):
    def __init__(self, config,  num_labels, hidden_size):
        super(BertSoftmaxForNer, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.softmax = nn.Softmax(dim=2)
        # self.init_weights()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None,labels=None):
        outputs = self.bert(input_ids = input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        sequence_output = outputs[0]
        class_probs = self.classifier(sequence_output)
        class_probs = self.softmax(class_probs)
        
        loss_fct = CrossEntropyLoss(ignore_index=0)
        # print(class_probs.view(-1, self.num_labels).shape)
        # print(labels.shape)

        loss = loss_fct(class_probs.view(-1, self.num_labels), labels)   

        return (class_probs, loss)

text = "lalalala"
labels = torch.tensor([x for x in range(5)])

tokens = tokenizer(text, return_tensors="pt")

ids = tokens.input_ids.tolist()
print(tokenizer.convert_ids_to_tokens(ids[0], skip_special_tokens=False))

model = BertSoftmaxForNer(config, 19, 768)
prediction = model.forward(tokens.input_ids, labels=labels)

prediction = model.forward(tokens.input_ids, labels=labels)
print(torch.argmax(prediction[0], dim = 2))

model.train()
for i in tqdm(range(100)):
    prediction = model.forward(tokens.input_ids, labels=labels)
    prediction[1].backward()
    

prediction = model.forward(tokens.input_ids, labels=labels)
print(torch.argmax(prediction[0], dim = 2))

# print(prediction)

['[CLS]', 'la', '##lala', '##la', '[SEP]']
tensor([[ 0,  0, 18, 11,  0]])


100%|██████████| 100/100 [00:14<00:00,  6.78it/s]

tensor([[ 0,  0, 10, 10,  4]])


In [ ]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
        {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,},
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=0.2, eps=1e-08)